In [59]:
from typing import Tuple, Callable

In [1]:
import torch

In [2]:
from torch import nn
from torch.nn import functional as F

In [17]:
class SimpleEmbedder(nn.Module):
    
    def __init__(self, input_channels, channels, embedding_size,
                 kernal_size=7, use_bias=False):
        super().__init__()
        self.separable_conv = nn.Sequential(
            nn.Conv2d(input_channels, channels,
                      kernel_size=(kernal_size, 1), padding=1,
                      bias=use_bias),
            nn.Conv2d(channels, channels,
                      kernel_size=(1, kernal_size), padding=2,
                      bias=use_bias),
        )
        self.linear = nn.Linear(channels * 32 * 32, embedding_size,
                                bias=use_bias)
    def forward(self, image: torch.Tensor) -> torch.Tensor:
        hidden = F.relu(self.separable_conv(image))
        return self.linear(torch.flatten(hidden, 1))

In [18]:
simple_embedder = SimpleEmbedder(3, 32, 128)

In [20]:
simple_embedder(torch.randn(1, 3, 32, 32)).shape

torch.Size([1, 128])

In [50]:
class RNNRouter(nn.Module):
    
    def __init__(self, hidden_size, operations, use_input_embedding=True,
                 weight_tie_output_layer=True, use_bias=True) -> None:
        '''
        Args:
            hidden_size:
            operations:
            use_input_embedding:
            weight_tie_output_layer:
            bias:
        Returns:
        '''
        super().__init__()
        self.hidden_size = hidden_size
        self.use_input_embedding = use_input_embedding
        self.operation_embedding = nn.Parameter(
            torch.empty(operations, hidden_size).uniform_(-0.1, 0.1)
        )
        if use_input_embedding:
            self.input_embedding = torch.empty(1, hidden_size).uniform_(-0.1, 0.1)
        self.rnn_cell = nn.GRUCell(hidden_size, hidden_size,
                                   bias=use_bias)
        self.output_layer = nn.Linear(hidden_size, operations,
                                bias=use_bias)
    
    def forward(self, hidden_state, last_decision=None) -> Tuple[torch.Tensor, torch.Tensor]:
        batch_size = hidden_state.size(0)
        if last_decision is None:
            if self.use_input_embedding:
                input_state = self.input_embedding.expand(batch_size, -1)
            else:
                input_state = torch.zeros(batch_size, self.hidden_size).uniform_(-0.1, 0.1)
        else:
            input_state = last_decision @ self.operations_embeddings
        hidden = self.rnn_cell(input_state, hidden_state)
        return hidden, self.output_layer(hidden)

In [55]:
rnn_router = RNNRouter(128, 10)

In [58]:
rnn_router(torch.randn(1, 128))[0].shape

torch.Size([1, 128])


torch.Size([1, 128])

In [ ]:
def beam_search(routing_function: Callable, beams, temperature: float = 1.):
    batch_size =
    batches_operations_reshape = (-1, beams)
    batches_beams_operations_reshape

In [ ]:
class BeamSearch:
    
    def __init__(self, beams: int,
                 operations: int, maximum_depth: int,
                 preserve_zeros_grad: bool = True):
        self.rout